In [5]:
import csv

# 1. Leer el archivo CSV
def leer_csv(archivo):
    datos = []
    with open(archivo, mode='r', encoding='utf-8') as file:
        lector = csv.reader(file)
        encabezados = next(lector)  # Leer encabezados
        for fila in lector:
            datos.append(fila)
    return encabezados, datos

encabezados, datos = leer_csv('diabetes.csv')


In [6]:
# 2. Contar filas y columnas
def contar_filas_columnas(datos, encabezados):
    num_filas = len(datos)
    num_columnas = len(encabezados)
    return num_filas, num_columnas

num_filas, num_columnas = contar_filas_columnas(datos, encabezados)
print(f"\n2. El dataset tiene {num_filas} filas y {num_columnas} columnas")
print("   Encabezados:", encabezados)


2. El dataset tiene 768 filas y 9 columnas
   Encabezados: ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']


In [7]:
# 3. Imprimir primeros 5 registros
print("\n3. Primeros 5 registros:")
print(encabezados)
for i in range(5):
    print(datos[i])


3. Primeros 5 registros:
['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
['6', '148', '72', '35', '0', '33.6', '0.627', '50', '1']
['1', '85', '66', '29', '0', '26.6', '0.351', '31', '0']
['8', '183', '64', '0', '0', '23.3', '0.672', '32', '1']
['1', '89', '66', '23', '94', '28.1', '0.167', '21', '0']
['0', '137', '40', '35', '168', '43.1', '2.288', '33', '1']


In [8]:
# 4. Valores únicos en la columna 'Outcome' (columna 8)
col_outcome = 8
valores_unicos = set(fila[col_outcome] for fila in datos)
print(f"\n4. Valores únicos en la columna 'Outcome': {valores_unicos}")


4. Valores únicos en la columna 'Outcome': {'0', '1'}


In [9]:
# 5. Calcular la media de la columna 'Glucose' (columna 1)
col_glucosa = 1
suma = 0
valores_validos = 0

for fila in datos:
    try:
        valor = float(fila[col_glucosa])
        suma += valor
        valores_validos += 1
    except ValueError:
        continue

media_glucosa = suma / valores_validos if valores_validos > 0 else 0
print(f"\n5. Media de glucosa: {media_glucosa:.2f} mg/dL")



5. Media de glucosa: 120.89 mg/dL


In [10]:
# 6. Encontrar máximo y mínimo en la columna 'BMI' (columna 5)
col_bmi = 5
valores_bmi = []

for fila in datos:
    try:
        valores_bmi.append(float(fila[col_bmi]))
    except ValueError:
        continue

max_bmi = max(valores_bmi)
min_bmi = min(valores_bmi)
print(f"\n6. BMI - Máximo: {max_bmi:.2f}, Mínimo: {min_bmi:.2f}")


6. BMI - Máximo: 67.10, Mínimo: 0.00


In [11]:
# 7. Distribución de edades y su relación con diabetes
def distribucion_edad_diabetes(datos, col_edad=7, col_outcome=8):
    edades_diabetes = []
    edades_no_diabetes = []
    
    for fila in datos:
        try:
            edad = int(fila[col_edad])
            if fila[col_outcome] == '1':
                edades_diabetes.append(edad)
            else:
                edades_no_diabetes.append(edad)
        except ValueError:
            continue
    
    # Calcular promedios
    prom_diabetes = sum(edades_diabetes)/len(edades_diabetes) if edades_diabetes else 0
    prom_no_diabetes = sum(edades_no_diabetes)/len(edades_no_diabetes) if edades_no_diabetes else 0
    
    # Calcular porcentaje de diabetes por grupo de edad
    grupos = {'<30': 0, '30-40': 0, '40-50': 0, '50-60': 0, '60+': 0}
    total_por_grupo = {'<30': 0, '30-40': 0, '40-50': 0, '50-60': 0, '60+': 0}
    
    for edad, outcome in [(int(fila[col_edad]), fila[col_outcome]) for fila in datos if fila[col_edad].isdigit()]:
        if edad < 30:
            grupo = '<30'
        elif 30 <= edad < 40:
            grupo = '30-40'
        elif 40 <= edad < 50:
            grupo = '40-50'
        elif 50 <= edad < 60:
            grupo = '50-60'
        else:
            grupo = '60+'
            
        total_por_grupo[grupo] += 1
        if outcome == '1':
            grupos[grupo] += 1
    
    print("\n7. Análisis sorprendente: Relación entre edad y diabetes")
    print(f"   Edad promedio con diabetes: {prom_diabetes:.1f} años")
    print(f"   Edad promedio sin diabetes: {prom_no_diabetes:.1f} años")
    print("\n   Porcentaje de diabetes por grupo de edad:")
    for grupo in grupos:
        total = total_por_grupo[grupo]
        if total > 0:
            porcentaje = (grupos[grupo] / total) * 100
            print(f"   {grupo}: {porcentaje:.1f}% ({grupos[grupo]}/{total})")

distribucion_edad_diabetes(datos)

# Función adicional: Mostrar histograma ASCII de una columna
def histograma_ascii(datos, col_idx, titulo, bins=10):
    valores = []
    for fila in datos:
        try:
            valores.append(float(fila[col_idx]))
        except ValueError:
            continue
    
    if not valores:
        print(f"No hay valores válidos para {titulo}")
        return
    
    min_val = min(valores)
    max_val = max(valores)
    rango = max_val - min_val
    if rango == 0:
        return
    
    # Crear bins
    histograma = [0] * bins
    for val in valores:
        indice = min(int((val - min_val) / rango * bins), bins - 1)
        histograma[indice] += 1
    
    max_freq = max(histograma)
    if max_freq == 0:
        return
    
    print(f"\nHistograma de {titulo} (de {min_val:.1f} a {max_val:.1f}):")
    for i in range(bins):
        bin_start = min_val + (i * rango / bins)
        bin_end = min_val + ((i + 1) * rango / bins)
        bar_length = int(histograma[i] / max_freq * 50)
        print(f"  [{bin_start:.1f}-{bin_end:.1f}): {'#' * bar_length} {histograma[i]}")

# Mostrar histograma de BMI
histograma_ascii(datos, 5, "BMI")


7. Análisis sorprendente: Relación entre edad y diabetes
   Edad promedio con diabetes: 37.1 años
   Edad promedio sin diabetes: 31.2 años

   Porcentaje de diabetes por grupo de edad:
   <30: 21.2% (84/396)
   30-40: 46.1% (76/165)
   40-50: 55.1% (65/118)
   50-60: 59.6% (34/57)
   60+: 28.1% (9/32)

Histograma de BMI (de 0.0 a 67.1):
  [0.0-6.7): ## 11
  [6.7-13.4):  0
  [13.4-20.1): ## 15
  [20.1-26.8): ############################# 156
  [26.8-33.5): ################################################## 268
  [33.5-40.3): ######################################### 224
  [40.3-47.0): ############## 78
  [47.0-53.7): ## 12
  [53.7-60.4):  3
  [60.4-67.1):  1
